In [ ]:
from __future__ import print_function
import os
import sys
sys.path.append(os.path.join(os.environ['ITHEMAL_HOME'], 'learning', 'pytorch'))

In [ ]:
%matplotlib inline

import numpy as np
import torch
import data.data_cost as dt
import common_libs.utilities as ut
import models.train as tr
import models.graph_models as md
import models.losses as ls
import random

In [ ]:
data = dt.load_dataset('/home/ithemal/ithemal/learning/pytorch/inputs/embeddings/code_delim.emb', data_savefile='/home/ithemal/ithemal/learning/pytorch/saved/time_skylake_1217.data')

ys = np.array([d.y for d in data.train])
mean = ys.mean()
std = ys.std()

In [ ]:
embedding_size = data.final_embeddings.shape[1]
model = md.GraphNN(embedding_size, 256, 1, False)
model.set_learnable_embedding(mode = 'none', dictsize = max(data.word2id) + 1, seed = data.final_embeddings)

loss_ema = 1

In [ ]:
adam = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [ ]:
# state_dict = torch.load('../saved/edges_none_01-05-19_22:26:09.mdl')
# model.load_state_dict(state_dict['model'])

In [ ]:
loss_fn = torch.nn.MSELoss()
def pred_loss(item, train=True, backward=True):
    pred = model(item)
    model.remove_refs(item)
    y = np.log(item.y)
    loss = torch.sqrt(loss_fn(pred, torch.FloatTensor([y]).squeeze())) / y
    if backward:
        loss.backward()
    return pred.item(), loss.item()

In [ ]:
import heapq
maxnorm = lambda vecs: max((p.data.norm(2).item() for p in vecs))

while True:
    adam.zero_grad()
    item = random.choice(data.train)
    pred, loss = pred_loss(item)
    adam.step()

    loss_ema = 0.999 * loss_ema + 0.001 * loss
    max_param = maxnorm(model.parameters())
    max_grad = maxnorm(filter(lambda p: p is not None, map(lambda p: p.grad, model.parameters())))

    report = 'EMA: {:.2f}, Loss: {:.3f}, Max param: {:.2f}, Max Grad: {:.2f} ({:.2f}, {:.2f})'.format(
        loss_ema, loss, max_param, max_grad, pred, np.log(item.y))
    print(report + ' '*20, end='\r')

In [ ]:
train = tr.Train(model, data, 4)
train.loss_fn = ls.mse_loss
train.print_fn = train.print_final
train.correct_fn = train.correct_regression
train.num_losses = 1

In [ ]:
actual, predicted = train.validate('/tmp/foo')
actual = np.array(actual)
predicted = np.exp(np.array(predicted))

In [ ]:
actual_t = torch.Tensor(actual)
predicted_t = torch.Tensor(predicted)

In [ ]:
(np.abs(actual - predicted) / actual).mean()

In [ ]:
for (a, p) in zip(actual_t, predicted_t):
    train.correct_regression(a, p)

In [ ]:
train.correct / float(len(data.test))

In [ ]:
model.lstm_token.weight_ih_l0.norm()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

print(len(ut.get_sym_dict()[0]))

class AlexDagRnn(nn.Module):
    def __init__(self, embedding_size, hidden_size):
        super(AlexDagRnn, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        
        sym_dict, _ = ut.get_sym_dict()
        self.max_tok = len(sym_dict) + 1
        
        embedding = nn.Embedding(self.max_tok, self.embedding_size)
        initrange = 0.5 / self.embedding_size
        embedding.weight.data.uniform_(-initrange, initrange)
        self.final_embeddings = embedding
        
        self.linear_token_1 = nn.Linear(self.embedding_size, self.hidden_size)
        self.linear_token_2 = nn.Linear(3 * self.hidden_size, self.hidden_size)
        self.lstm_ins = nn.LSTM(self.hidden_size, self.hidden_size)
        
    def embed_instr(self, instr):
        op = self.final_embeddings(instr.opcode)
        srcs = [self.final_embeddings(min(src, self.max_tok)) for src in instr.srcs] + [torch.zeros(self.embedding_size)]
        
    def forward(self, datum):
        embeddings = {}